In [1]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import statsmodels.api as sm

In [2]:
from TDSCoinbaseData import TDSCoinbaseData
from TDSTickGenerator import TDSTickGenerator
from TDSTransactionTracker import TDSTransactionTracker
import logging
logging.getLogger().setLevel(level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [3]:
# instantiate our data interface
cb_obj = TDSCoinbaseData(cache_path='../data', notebook_logging=True)
# Set the start and end date
start_date = '20201231'
end_date = '20201231'

# List all of the products to be used in the strategy. Be sure to list all products you may use, these can't be updated later
products = ['BTC-USD', 'ETH-USD', 'LTC-USD']

In [4]:
MAX_TRAINNING_SIZE = 360
SIZE_LIMIT_REACHED = False

In [5]:
def append_ticks(tick, usd_btc, eth_btc, ltc_btc, scale=1000, max_r=SIZE_LIMIT_REACHED, max_size=MAX_TRAINNING_SIZE):
    x = tick.p.btc_usd
    y = tick.p.eth_usd
    z = tick.p.ltc_usd
    usd_btc = usd_btc.append({'open': scale/x.open, 'close': scale/x.close, 
                      'volume': x.volume, 'datetime': tick.datetime}, ignore_index=True)
    eth_btc = eth_btc.append({'open': scale * y.open/x.open, 'close': scale *  y.close/x.close, 
                      'volume': y.volume, 'datetime': tick.datetime}, ignore_index=True)
    ltc_btc = ltc_btc.append({'open': scale * z.open/x.open, 'close': scale * z.close/x.close, 
                      'volume': z.volume, 'datetime': tick.datetime}, ignore_index=True)
    # Checks if size limit has been reached
    if (SIZE_LIMIT_REACHED or len(usd_btc.index) > max_size) :
        usd_btc = usd_btc.iloc[1:]
        max_r = True
    if (SIZE_LIMIT_REACHED or len(eth_btc.index) > max_size) :
        eth_btc = eth_btc.iloc[1:]
        max_r = True
    if (SIZE_LIMIT_REACHED or len(ltc_btc.index) > max_size) :
        ltc_btc = ltc_btc.iloc[1:]
        max_r = True
    return(usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED)

In [6]:
def calculate_time(pddatetime, numtime, plus=True):
    if plus:
        return pddatetime.to_pydatetime() + timedelta(minutes=numtime)
    else: 
        return pddatetime.to_pydatetime() - timedelta(minutes=numtime)

# frame -- dataframe
# minute -- the number of minutes to be predicted
def predict(frame, minute=3) :
    frame = frame.set_index('datetime')
    y = frame['open'].resample('1T').mean()
    l = frame['close'].resample('1T').mean()
    v = frame['volume'].resample('1T').mean()
    merged = pd.merge(y, l, on='datetime', how="outer")
    frame = pd.merge(merged, v, on='datetime', how="outer")

    mod = sm.tsa.VARMAX(frame[['open', 'close', 'volume']], order=(1,0), error_cov_type='diagonal')
    results = mod.fit(maxiter=50, disp=False)
#     print(results.summary().tables[1])
    newtime = calculate_time(tick.datetime, minute)
    pred = results.get_prediction(start=pd.to_datetime(frame.index[0]), end=newtime, dynamic=False)
    pred_ci = pred.conf_int()
#     print(pred.predicted_mean)
    return pred

# Call this function for btc prediction
#     minute -- the number of minutes to be predicted
# def predict_btc(minute) :
#     return predict(usd_btc, minute)
    
# def predict_eth(minute) :
#     return predict(eth_btc, minute)

In [7]:
start_date = '20201231'
end_date = '20201231'

products = ['BTC-USD', 'ETH-USD', 'LTC-USD']

# Instantiate our transaction tracker. As required, we start with 1.0 BTC
trans_tracker = TDSTransactionTracker(start_date, end_date, holdings={'BTC' : 1.0})

# Instantiate a tick generator
tick_gen = TDSTickGenerator(cb_obj, products, start_date, end_date, interval=60)
tick = tick_gen.get_tick()

# Dataframe for history
d = {'open': [], 'close': [], 'volume': [], 'datetime': []}
usd_btc = pd.DataFrame(data=d)
eth_btc = pd.DataFrame(data=d)
ltc_btc = pd.DataFrame(data=d)

#loads initial value for testing
for i in range(0, 150):
    tick = tick_gen.get_tick()
    (usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED) = append_ticks(tick, usd_btc, eth_btc, ltc_btc)

In [8]:
def get_coin_ratio(tick, holdings):
    btc = holdings.get("BTC") if "BTC" in holdings else 0
    eth = tick.p.eth_usd.close/tick.p.btc_usd.close * holdings.get("ETH") if "ETH" in holdings else 0
    ltc = tick.p.ltc_usd.close/tick.p.btc_usd.close * holdings.get("LTC") if "LTC" in holdings else 0
    usd = holdings.get("USD")/tick.p.btc_usd.close if "USD" in holdings else 0
    res = np.array([usd, eth, ltc, btc])
    return sum(res), res/sum(res)

In [9]:
def trade_ratio(curr_port, ideal_port, tick, trans_tracker, curr_hold_in_BTC):
    gap = (ideal_port - curr_port)[1:]
    gap_in_usd = gap * curr_hold_in_BTC * tick.p.btc_usd.close
    names = np.array(['ETH-USD', 'LTC-USD', 'BTC-USD'])
    names = names[np.argsort(gap_in_usd)]
    gap_in_usd = np.sort(gap_in_usd)
    # holdings = trans_tracker.get_holdings()
    
    for idx in range(len(names)):
        product = names[idx]
        change = gap_in_usd[idx]
        attr = getattr(tick.p, product.lower().replace('-', '_'))
        # curr_hold = trans_tracker.get_holdings().get(product) if product in 
        if change < 0:
            valid_vol = min(- change / attr.close, attr.volume * 0.5)
#             print(product[:3], trans_tracker.get_holdings().get(product[:3]))
            valid_vol = min(valid_vol, trans_tracker.get_holdings().get(product[:3]))
#             print(- change / attr.close, attr.volume * 0.5)
            trans_tracker.make_trade(tick, product, 'sell', valid_vol)
        elif change > 0:
            valid_amt = min(change, trans_tracker.get_holdings().get("USD"))
#             print(attr.volume)
            valid_amt = min(valid_amt, attr.volume * attr.close * 0.5)
#             print(valid_amt)
            trans_tracker.make_trade(tick, product, 'buy', valid_amt)
            
    # print(names, gap_in_usd)
    
#     btc_gap = (usd_val * b - btc) / tick.p.btc_usd.close
#     eth_gap = (usd_val * e - eth) / tick.p.eth_usd.close
#     ltc_gap = (usd_val * l - ltc) / tick.p.ltc_usd.close
    
#     if (btc_gap > 0):
#         trans_tracker.make_trade(tick, 'BTC-USD', 'sell', btc_gap / tick.p.btc_usd.close)
#     if (eth_gap > 0):
#         trans_tracker.make_trade(tick, 'ETH-USD', 'sell', eth_gap / tick.p.eth_usd.close)
#     if (ltc_gap > 0):
#         trans_tracker.make_trade(tick, 'LTC-USD', 'sell', ltc_gap / tick.p.ltc_usd.close)
        
#     if (btc_gap < 0):
#         trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(btc_gap, trans_tracker.get_holdings().get("USD")))
#     if (eth_gap < 0):
#         trans_tracker.make_trade(tick, 'ETH-USD', 'buy', min(eth_gap, trans_tracker.get_holdings().get("USD")))
#     if (ltc_gap < 0):
#         trans_tracker.make_trade(tick, 'LTC-USD', 'buy', min(ltc_gap, trans_tracker.get_holdings().get("USD")))

In [10]:
# Actual trading strategy

count = 0
while tick is not None and count < 20:
    
    # Get current prices 
    x_price = 1000 / tick.p.btc_usd.close
    y_price = 1000 * tick.p.eth_usd.close / tick.p.btc_usd.close
    z_price = 1000 * tick.p.ltc_usd.close / tick.p.btc_usd.close
    
    curr_hold_in_BTC, curr_port = get_coin_ratio(tick, trans_tracker.get_holdings())
    # Get predicted values
    pred_usd = predict(usd_btc, 1)
    pred_eth = predict(eth_btc, 1)
    pred_ltc = predict(ltc_btc, 1)
    
    # Find which one increases the most
    changes = np.array([(pred_usd.predicted_mean["close"][-1]-x_price)/x_price, 
               (pred_eth.predicted_mean["close"][-1]-y_price)/y_price, 
               (pred_ltc.predicted_mean["close"][-1]-z_price)/z_price, 
                       0])

    
    # interval = (pred_usd.conf_int()["upper close"][-1] - pred_usd.conf_int()["lower close"][-1]) / ratio
#     print(count, usd_btc["close"][len(usd_btc)-1], pred_usd.conf_int()["lower close"][-1], pred_usd.conf_int()["upper close"][-1])
#     if pred_usd.predicted_mean["close"][-1] - interval <= usd_btc["close"][len(usd_btc)-1] <= pred_usd.predicted_mean["close"][-1] + interval:
#         x_acc[0] += 1
#     else:
#         x_acc[1] += 1
    
    # Obtain ideal portfolio
    ideal_port = np.zeros_like(changes)
    ideal_port[changes.argmax(0)] = 1
    
    print(count, curr_port, ideal_port)
    trade_ratio(curr_port, ideal_port, tick, trans_tracker, curr_hold_in_BTC)
    
    (usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED) = append_ticks(tick, usd_btc, eth_btc, ltc_btc)
    tick = tick_gen.get_tick()
    count += 1

0 [0. 0. 0. 1.] [0. 0. 1. 0.]
1 [0.90701516 0.         0.09298484 0.        ] [0. 0. 0. 1.]
2 [0.         0.         0.06503451 0.93496549] [1. 0. 0. 0.]
3 [1. 0. 0. 0.] [0. 1. 0. 0.]
4 [0. 1. 0. 0.] [0. 0. 1. 0.]
5 [0.76731033 0.         0.23268967 0.        ] [1. 0. 0. 0.]
6 [0.86277448 0.         0.13722552 0.        ] [0. 1. 0. 0.]
7 [0.06437534 0.88782901 0.04779564 0.        ] [0. 1. 0. 0.]
8 [0.00000000e+00 1.00000000e+00 7.92459888e-18 0.00000000e+00] [0. 0. 0. 1.]
9 [0.         0.10247877 0.         0.89752123] [0. 0. 1. 0.]
10 [0.77628347 0.         0.22371653 0.        ] [0. 0. 0. 1.]
11 [0.         0.         0.13131633 0.86868367] [0. 0. 0. 1.]
12 [0. 0. 0. 1.] [0. 0. 0. 1.]
13 [0. 0. 0. 1.] [1. 0. 0. 0.]
14 [1. 0. 0. 0.] [0. 0. 1. 0.]
15 [0.94584379 0.         0.05415621 0.        ] [0. 0. 0. 1.]
16 [0.00000000e+00 0.00000000e+00 7.98901481e-18 1.00000000e+00] [0. 0. 1. 0.]
17 [0.38948773 0.         0.61051227 0.        ] [1. 0. 0. 0.]
18 [0.64910442 0.         0.35089558

In [11]:
trans_tracker.get_holdings()

{'BTC': 0.0, 'USD': 7613.321738014282, 'LTC': 161.19444545298103, 'ETH': 0.0}

In [13]:
get_coin_ratio(tick, trans_tracker.get_holdings())

(0.9749163884448557, array([0.27206502, 0.        , 0.72793498, 0.        ]))